# Clustering Crypto

In [220]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [221]:
# Load the crypto_data.csv dataset.
file_path = "crypto_data.csv"
crypto_df = pd.read_csv(file_path, index_col=0)
crypto_df.head(100)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
FRK,Franko,Scrypt,True,PoW,1.142732e+06,11235813
FRAC,FractalCoin,X11,True,PoW,NaN,1000000
FSTC,FastCoin,Scrypt,False,PoW,NaN,165888000
FTC,FeatherCoin,NeoScrypt,True,PoW,2.086691e+08,336000000


In [222]:
# Keep all the cryptocurrencies that are being traded.
print(crypto_df.shape)

(1252, 6)


In [223]:
crypto_df['IsTrading'].value_counts()

True     1144
False     108
Name: IsTrading, dtype: int64

In [224]:
crypto_df.drop(crypto_df.loc[crypto_df['IsTrading']==False].index, inplace=True)

In [225]:
crypto_df['IsTrading'].value_counts()

True    1144
Name: IsTrading, dtype: int64

In [226]:
crypto_df['Algorithm'].value_counts()

Scrypt            394
X11               182
SHA-256           121
X13                54
PoS                42
                 ... 
vDPOS               1
X11GOST             1
Cloverhash          1
Ouroboros           1
Green Protocol      1
Name: Algorithm, Length: 89, dtype: int64

In [227]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_df['Algorithm'].isnull().sum()

0

In [228]:
# Remove the "IsTrading" column. 
del crypto_df['IsTrading']

In [229]:
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [230]:
print(crypto_df.shape)

(1144, 5)


In [231]:
# Remove rows that have at least 1 null value.
crypto_df = crypto_df.dropna()

In [232]:
print(crypto_df.shape)

(685, 5)


In [233]:
# Keep the rows where coins are mined.
crypto_df.drop(crypto_df.loc[crypto_df['TotalCoinsMined'] == 0].index, inplace=True)

In [234]:
print(crypto_df.shape)

(533, 5)


In [235]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_name_df = pd.DataFrame(data = crypto_df, columns = ['CoinName'])
crypto_name_df

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex
ZEN,Horizen


In [236]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
del crypto_df['CoinName']

In [237]:
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [238]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df, columns=['Algorithm', 'ProofType'])
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [239]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled[0:1])

[[-0.11674788 -0.15286468 -0.0433555  -0.0433555  -0.0433555  -0.06137164
  -0.07523548 -0.0433555  -0.06137164 -0.06137164 -0.0433555  -0.0433555
  -0.19226279 -0.06137164 -0.09731237 -0.0433555  -0.11536024 -0.07523548
  -0.0433555  -0.0433555  -0.15176505 -0.0433555  -0.13105561 -0.0433555
  -0.0433555  -0.08695652 -0.0433555  -0.0433555  -0.0433555  -0.0433555
  -0.06137164 -0.0433555  -0.08695652 -0.08695652 -0.08695652 -0.0433555
  -0.13105561 -0.13827675 -0.13827675 -0.0433555  -0.06137164 -0.0433555
  -0.07523548 -0.1815096  -0.0433555  -0.0433555  -0.0433555  -0.07523548
  -0.15811388 -0.3145935  -0.0433555  -0.08695652 -0.07523548 -0.06137164
  -0.0433555   1.38873015 -0.0433555  -0.0433555  -0.06137164 -0.0433555
  -0.0433555  -0.0433555  -0.0433555  -0.0433555  -0.0433555  -0.0433555
  -0.0433555  -0.39836623 -0.0433555  -0.1815096  -0.0433555  -0.08695652
  -0.08695652 -0.10670145 -0.0433555  -0.0433555  -0.13105561 -0.0433555
  -0.0433555  -0.0433555  -0.0433555  -0.07523

### Deliverable 2: Reducing Data Dimensions Using PCA

In [240]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)

In [241]:
crypto_pca = pca.fit_transform(X_scaled)

In [242]:
# Create a DataFrame with the three principal components.
pca_df = pd.DataFrame(data=crypto_pca, columns=["PC1", "PC2", "PC3"], index=crypto_df.index)
pca_df.head(10)

,PC1,PC2,PC3
42,-0.327596,0.970432,-0.545005
404,-0.310966,0.970088,-0.545437
1337,2.284967,1.429764,-0.664805
BTC,-0.140754,-1.284903,0.197403
ETH,-0.148416,-1.983988,0.388344
LTC,-0.154703,-1.082472,0.033724
DASH,-0.401699,1.147879,-0.493415
XMR,-0.138877,-2.135038,0.436952
ETC,-0.146862,-1.984135,0.388331
ZEC,-0.134745,-2.046639,0.417253


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [243]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pca_df)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")


:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [244]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pca_df)

# Predict clusters
predictions = model.predict(pca_df)
pca_df['class'] = model.labels_

In [245]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df, crypto_name_df, pca_df], axis=1)
clustered_df

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
# YOUR CODE HERE

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
# YOUR CODE HERE

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(533, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,CoinName,PC1,PC2,PC3,class
42,Scrypt,PoW/PoS,4.199995e+01,42,42 Coin,-0.327596,0.970432,-0.545005,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,404Coin,-0.310966,0.970088,-0.545437,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,EliteCoin,2.284967,1.429764,-0.664805,0
BTC,SHA-256,PoW,1.792718e+07,21000000,Bitcoin,-0.140754,-1.284903,0.197403,1
ETH,Ethash,PoW,1.076842e+08,0,Ethereum,-0.148416,-1.983988,0.388344,1
LTC,Scrypt,PoW,6.303924e+07,84000000,Litecoin,-0.154703,-1.082472,0.033724,1
DASH,X11,PoW/PoS,9.031294e+06,22000000,Dash,-0.401699,1.147879,-0.493415,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,Monero,-0.138877,-2.135038,0.436952,1
ETC,Ethash,PoW,1.133597e+08,210000000,Ethereum Classic,-0.146862,-1.984135,0.388331,1
ZEC,Equihash,PoW,7.383056e+06,21000000,ZCash,-0.134745,-2.046639,0.417253,1


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [247]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC1",
    y="PC2",
    z="PC3",
    color="class",
    hover_name="CoinName",
    hover_data=['Algorithm'],
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


In [248]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "class"])

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,class]

In [249]:
# Print the total number of tradable cryptocurrencies.
print(f"There are {len(clustered_df)} tradable cryptocurrencies.")

There are 533 tradable cryptocurrencies.


In [250]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
scaled_columns = clustered_df[["TotalCoinSupply", "TotalCoinsMined"]]
scaled = MinMaxScaler().fit_transform(scaled_columns)

In [251]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
scaled_df = pd.DataFrame(scaled, columns=["TotalCoinSupply", "TotalCoinsMined"], index=clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
scaled_df["CoinName"] = clustered_df["CoinName"]

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
scaled_df["Class"] = clustered_df["class"]
scaled_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.005942,42 Coin,0
404,5.320000e-04,0.007002,404Coin,0
1337,3.141593e-01,0.035342,EliteCoin,0
BTC,2.100000e-05,0.005960,Bitcoin,1
ETH,0.000000e+00,0.006050,Ethereum,1
LTC,8.400000e-05,0.006006,Litecoin,1
DASH,2.200000e-05,0.005951,Dash,0
XMR,0.000000e+00,0.005960,Monero,1
ETC,2.100000e-04,0.006056,Ethereum Classic,1
ZEC,2.100000e-05,0.005950,ZCash,1


In [252]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
scaled_df.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply", hover_cols=["CoinName"], by="Class")


:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)